**<font color = black size = 5>All of the component in the data collection<font>**

In [1]:
# IPython Libraries for display and widgets
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

# Camera and Motor Interface for JetBot
from jetbot import Robot, Camera, bgr8_to_jpeg

# Python basic pakcages for image annotation
from uuid import uuid1
import os
import json

import glob
import datetime
import numpy as np
import cv2
import time

**<font size = 5>Make Directory for Data</font>**

In [2]:
DATASET_DIR = 'dataset_xy_new'
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created because they already exist')

Directories not created because they already exist


**<font size = 5>Get Camera Image And Button</font>**

In [3]:
camera = Camera()

image_widget = widgets.Image(format='jpeg', width=224, height=224)
target_widget = widgets.Image(format='jpeg', width=224, height=224)

# x_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='x')
# y_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='y')

leftBottom = widgets.FloatSlider(min =-2.0, max=2.0, step=0.01, description='leftButtom')
leftTop = widgets.FloatSlider(min =-2.0, max=2.0, step=0.01, description='leftTop')
rightBottom = widgets.FloatSlider(min =-2.0, max=2.0, step=0.01, description='rightButtom')
rightTop = widgets.FloatSlider(min =-2.0, max=2.0, step=0.01, description='rightTop')                               
                                
def display_xy(camera_image):
    image = np.copy(camera_image)
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.GaussianBlur(image, (5,5), 0)
    image = cv2.GaussianBlur(image, (5,5), 0)
    image = cv2.Canny(image, 170, 190)
    
#     rho = 1
#     theta = np.pi/180
#     threshold = 1
#     min_line_length = 10
#     max_line_gap = 1
    
#     line_image = np.copy(image) * 0
#     lines = cv2.HoughLinesP(image, rho, theta, threshold, min_line_length, max_line_gap)
#     for x1,y1,x2,y2 in lines[0]:
#         cv2.line(line_image,(x1,y1), (x2,y2),(255, 0, 0), 2)
#     image = cv2.addWeighted(image, 0.8, line_image, 1, 0) 
    
    
        
    # x = x_slider.value
    # y = y_slider.value
    
    LB = leftBottom.value
    LT = leftTop.value
    RB = rightBottom.value
    RT = rightTop.value
                        
    # x = int(x * 224 / 2 + 112)
    # y = int(y * 224 / 2 + 112) 
                               
    LB = int (LB * 224 / 2 + 112) 
    LT = int (LT * 224 / 2 + 112)                            
    RB = int (RB * 224 / 2 + 112)                            
    RT = int (RT * 224 / 2 + 112)     
    CB = int((LB + RB) / 2)
    CT = int((LT + RT) / 2)
    CC = int((CB + CT) / 2)
                               
    image = cv2.line(image, (LB, 224), (LT, 0), (255, 0 ,0), 2)
    image = cv2.line(image, (RB, 224), (RT, 0), (255, 0, 0), 2)
    image = cv2.line(image, (CB, 224), (CT, 0), (255, 0, 0), 2)
    image = cv2.circle(image, (CC, 112),8, (255, 0, 0), 2)   
    
#     image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
#     image = cv2.circle(image, (112, 224), 8, (0, 0,255), 3)
#     image = cv2.line(image, (x,y), (112,224), (255,0,0), 3)
    
    
    
    
    jpeg_image = bgr8_to_jpeg(image)
    # print(x, y)
    return jpeg_image

time.sleep(1)
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_xy)

**<font size = 5 color = blue> Start to Collect the Data(Including the number of picture)</font>**

In [4]:
button_layout = widgets.Layout(width='128px', height='64px')
button = widgets.Button(description='add', button_style='primary', layout=button_layout)
count = widgets.IntText(value=len(os.listdir(DATASET_DIR)),layout=button_layout)

In [5]:
def xy_uuid(x, y):
    return 'xy_%03d_%03d_%s' % (x * 224 / 2 + 224 / 2, y * 224 / 2 + 224 / 2, uuid1())

def get_this_frame():
    uuid = xy_uuid((leftBottom.value + leftTop.value + rightBottom.value + rightTop.value) / 4, 0)
    image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image_widget.value)
    count.value=len(os.listdir(DATASET_DIR))

In [6]:
button.on_click(lambda x:get_this_frame())
# display(widgets.HBox([image_widget, target_widget]))
display(widgets.HBox([image_widget, target_widget]))
# display(target_widget)
display(widgets.HBox([widgets.VBox([leftTop, leftBottom]), widgets.VBox([rightTop, rightBottom])]))
display(widgets.HBox([button, count]))

In [3]:
picture = len(os.listdir(DATASET_DIR))
print("Total pictures are " + str(picture))

Total pictures are 650


In [148]:
camera.stop()

**<font size = 5>Use Zip to low store total directory</font>**

In [1]:
DATASET_DIR = 'dataset_xy_new'


!zip -r -q {DATASET_DIR}.zip {DATASET_DIR}

In [29]:
import cv2
import os
import numpy as np

ORIGINAL_DIR = 'dataset_xy'
DATASET_DIR = 'dataset_xy_reduce'
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print("The directory is exist!!")

    
# img = cv2.imread("dataset_xy/xy_033_171_45308aac-bdf7-11ec-be0d-16f63a1aa8c9.jpg", 0)
# img = cv2.medianBlur(img, 5)
# img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, -1)
# cv2.imwrite(DATASET_DIR + "/"  + "xy_033_171_45308aac-bdf7-11ec-be0d-16f63a1aa8c9" + ".jpg", img)    

kernel = np.ones((3,3), np.uint8)
for image in os.listdir(ORIGINAL_DIR):
    # pass
    img = cv2.imread(ORIGINAL_DIR + "/" + image, 0)
    img = cv2.medianBlur(img, 5)
    img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, -1)
    
    img = cv2.erode(img, kernel, iterations = 1)
    img = cv2.dilate(img, kernel, iterations = 1)
    
    cv2.imwrite(DATASET_DIR + "/" + image, img)
  

The directory is exist!!
